# 실험 실행

Azure Machine Learning SDK를 사용하여 메트릭을 기록하고 출력을 생성하는 코드 실험을 실행할 수 있습니다. 이 실행 과정은 Azure Machine Learning에서 수행하는 대다수 기계 학습 작업의 핵심 요소입니다.

## 작업 영역에 연결합니다.

Azure Machine Learning 작업 영역 내에서 모든 실험 및 관련 리소스를 관리합니다. 대부분의 경우에는 JSON 구성 파일에 작업 영역 구성을 저장해야 합니다. 이렇게 하면 Azure 구독 ID 등의 세부 정보를 기억할 필요 없이 더욱 쉽게 다시 연결할 수 있습니다. Azure Portal의 작업 영역 블레이드에서 JSON 구성 파일을 다운로드할 수 있습니다. 그러나 작업 영역 내에서 컴퓨팅 인스턴스를 사용 중이라면 구성 파일은 루트 폴더에 이미 다운로드된 상태입니다.

아래 코드는 구성 파일을 사용하여 작업 영역에 연결합니다.

> **참고**: Azure 구독에 인증된 세션을 아직 설정하지 않은 경우에는 링크를 클릭하고 인증 코드를 입력한 다음 Azure에 로그인하여 인증하라는 메시지가 표시됩니다.

In [ ]:
import azureml.core
from azureml.core import Workspace

# 저장된 구성 파일에서 작업 영역 로드
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 실험 실행

데이터 과학자가 수행해야 하는 가장 기본적인 작업 중 하나는 데이터를 처리하고 분석하는 실험을 만들고 실행하는 것입니다. 이 연습에서는 Azure ML *실험*을 사용하여 Python 코드를 실행하고 데이터에서 추출된 값을 기록하는 방법을 알아봅니다. 여기서는 당뇨병 진단을 받은 환자의 세부 정보가 포함된 간단한 데이터 세트를 사용합니다. 실험을 실행하여 데이터를 살펴본 다음 통계, 시각화 및 데이터 샘플을 추출합니다. 여기서 사용할 대다수 코드는 데이터 탐색 프로세스에서 실행할 수 있는 코드와 같은 매우 일반적인 Python 코드입니다. 그러나 여기에 코드를 몇 줄만 추가하면 Azure ML *실험*을 사용하여 실행 세부 정보를 기록하도록 수정할 수 있습니다.

In [ ]:
from azureml.core import Experiment
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

# 작업 영역에서 Azure ML 실험 만들기
experiment = Experiment(workspace=ws, name="mslearn-diabetes")

# 실험에서 데이터 로깅을 시작하여 실험 실행에 대한 참조 가져오기
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# 로컬 파일에서 데이터 로드
data = pd.read_csv('data/diabetes.csv')

# 행 수를 계산하고 결과 로깅
row_count = (len(data))
run.log('observations', row_count)
print('Analyzing {} rows of data'.format(row_count))

# 당뇨병 환자와 기타 환자의 수 그리기/기록
diabetic_counts = data['Diabetic'].value_counts()
fig = plt.figure(figsize=(6,6))
ax = fig.gca()    
diabetic_counts.plot.bar(ax = ax) 
ax.set_title('Patients with Diabetes') 
ax.set_xlabel('Diagnosis') 
ax.set_ylabel('Patients')
plt.show()
run.log_image(name='label distribution', plot=fig)

# 개별 임신 횟수 기록
pregnancies = data.Pregnancies.unique()
run.log_list('pregnancy categories', pregnancies)

# 숫자 열의 요약 통계 기록
med_columns = ['PlasmaGlucose', 'DiastolicBloodPressure', 'TricepsThickness', 'SerumInsulin', 'BMI']
summary_stats = data[med_columns].describe().to_dict()
for col in summary_stats:
    keys = list(summary_stats[col].keys())
    values = list(summary_stats[col].values())
    for index in range(len(keys)):
        run.log_row(col, stat=keys[index], value = values[index])
        
# 데이터 샘플을 저장하고 실험 출력에 업로드
data.sample(100).to_csv('sample.csv', index=False, header=True)
run.upload_file(name='outputs/sample.csv', path_or_stream='./sample.csv')

# 실행 완료
run.complete()

## 실행 세부 정보 확인

Jupyter Notebook에서 **RunDetails** 위젯을 사용하면 실행 세부 정보의 시각화를 확인할 수 있습니다.

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

### Azure Machine Learning Studio에서 추가 세부 정보 확인

**RunDetails** 위젯에는 Azure Machine Learning Studio에서 **실행 세부 정보를 확인**할 수 있는 링크가 포함되어 있습니다. 이 링크를 클릭하면 새 브라우저 탭이 열리고 실행 세부 정보가 표시됩니다. [Azure Machine Learning Studio](https://ml.azure.com)를 열고 **실험** 페이지에서 실행을 찾아도 됩니다. Azure Machine Learning Studio에서 실행을 확인할 때는 다음 사항에 유의하세요.

- **세부 정보** 탭에는 실험 실행의 일반 속성이 포함되어 있습니다.
- **메트릭** 탭에서는 기록된 메트릭을 선택하여 표나 차트로 표시할 수 있습니다.
- **이미지** 탭에서는 실험에서 기록된 이미지나 그림(여기서는 *레이블 분포* 그림)을 선택하여 확인할 수 있습니다.
- **자식 실행** 탭에는 자식 실행의 목록이 표시됩니다. 이 실험에서는 자식 실행이 없습니다.
- **출력 + 로그** 탭에는 실험에서 생성된 출력 파일이나 로그 파일이 표시됩니다.
- **스냅샷** 탭에는 실험 코드가 실행된 폴더의 모든 파일(여기서는 이 Notebook과 같은 폴더에 있는 모든 파일)이 포함되어 있습니다.
- **설명** 탭은 실험에서 생성된 모델 설명을 표시하는 데 사용됩니다. 여기서는 생성된 설명이 없습니다.
- **공정성** 탭은 기계 학습 모델의 공정성을 평가하는 데 도움이 되는 예측 성능 차이(여기서는 없음)의 시각화에 사용됩니다.

### SDK를 사용하여 실험 세부 정보 검색

앞에서 실행한 코드의 **run** 변수는 **Run** 개체의 인스턴스입니다. Run 개체는 Azure Machine Learning에서 진행하는 실험의 개별 실행에 대한 참조입니다. 이 참조를 사용하여 실험 및 해당 출력 관련 정보를 가져올 수 있습니다.

In [ ]:
import json

# 로깅된 메트릭 가져오기
print("Metrics:")
metrics = run.get_metrics()
for metric_name in metrics:
    print(metric_name, ":", metrics[metric_name])

# 출력 파일 가져오기
print("\nFiles:")
files = run.get_file_names()
for file in files:
    print(file)

실험에서 생성된 파일은 **download_file** 메서드를 사용해 개별적으로 다운로드할 수도 있고 **download_files** 메서드를 사용해 여러 파일을 검색한 다음 다운로드할 수도 있습니다. 다음 코드는 실행의 **output** 폴더에 있는 모든 파일을 다운로드합니다.

In [ ]:
import os

download_folder = 'downloaded-files'

# "outputs" 폴더에 파일 다운로드
run.download_files(prefix='outputs', output_directory=download_folder)

# 파일이 다운로드되었는지 확인
for root, directories, filenames in os.walk(download_folder): 
    for filename in filenames:  
        print (os.path.join(root,filename))

실험 실행의 문제를 해결해야 하는 경우 **get_details** 메서드를 사용해 실행의 기본 세부 정보를 검색할 수도 있고, **get_details_with_logs** 메서드를 사용하여 실행 세부 정보와 실행에서 생성된 로그 파일의 내용을 검색할 수도 있습니다.

In [ ]:
run.get_details_with_logs()

세부 정보에는 실험을 실행한 컴퓨팅 대상 관련 정보, 그리고 실험을 시작 및 종료한 날짜와 시간이 포함됩니다. 또한 실험 코드가 포함된 Notebook(이 Notebook)이 복제된 Git 리포지토리에 있으므로 해당 리포지토리, 분기, 상태 관련 세부 정보도 실행 기록에 기록됩니다.

여기서 세부 정보의 **logFiles** 항목은 로그 파일이 생성되지 않았음을 나타냅니다. 방금 실행한 실험과 같은 인라인 실험에서는 대개 로그 파일이 생성되지 않습니다. 하지만 스크립트를 실험으로 실행하는 경우에는 결과가 달라집니다. 다음에는 이러한 실험에 대해 살펴보겠습니다.

## 실험 스크립트 실행

이전 예제에서는 이 Notebook에서 실험을 인라인으로 실행했습니다. 실험을 더 유동적으로 실행하려는 경우 별도의 실험용 스크립트를 작성하여 이 스크립트에 필요한 다른 파일과 함께 폴더에 저장한 다음, Azure ML을 사용하여 해당 폴더의 스크립트를 기준으로 실험을 실행하면 됩니다.

먼저 실험 파일용 폴더를 만들고 이 폴더에 데이터를 복사해 보겠습니다.

In [ ]:
import os, shutil

# 실험 파일용 폴더 만들기
folder_name = 'diabetes-experiment-files'
experiment_folder = './' + folder_name
os.makedirs(folder_name, exist_ok=True)

# 실험 폴더에 데이터 파일 복사
shutil.copy('data/diabetes.csv', os.path.join(folder_name, "diabetes.csv"))

실험용 코드가 포함된 Python 스크립트를 만들어 실험 폴더에 저장합니다.

> **참고**: 다음 셀을 실행하면 스크립트 파일이 *작성*만 되며 실행되지는 않습니다.

In [ ]:
%%writefile $folder_name/diabetes_experiment.py
from azureml.core import Run
import pandas as pd
import os

# 실험 실행 컨텍스트 가져오기
run = Run.get_context()

# 당뇨병 데이터 세트 로드
data = pd.read_csv('diabetes.csv')

# 행 수를 계산하고 결과 로깅
row_count = (len(data))
run.log('observations', row_count)
print('Analyzing {} rows of data'.format(row_count))

# 레이블 수 계산 및 기록
diabetic_counts = data['Diabetic'].value_counts()
print(diabetic_counts)
for k, v in diabetic_counts.items():
    run.log('Label:' + str(k), v)
      
# 출력 폴더에 데이터 샘플 저장(샘플은 자동으로 업로드됨)
os.makedirs('outputs', exist_ok=True)
data.sample(100).to_csv("outputs/sample.csv", index=False, header=True)

# 실행 완료
run.complete()

이 코드는 이전에 사용했던 인라인 코드의 간단한 버전이며 다음과 같은 작업을 수행합니다.
- `Run.get_context()` 메서드를 사용하여 스크립트 실행 시에 실험 실행 컨텍스트를 검색합니다.
- 스크립트가 있는 폴더에서 당뇨병 데이터를 로드합니다.
- **outputs** 폴더를 만들고 이 폴더에 샘플 파일을 작성합니다. 이 폴더는 실험 실행에 자동으로 업로드됩니다.

이제 실험을 실행할 준비가 거의 완료되었습니다. 스크립트를 실행하려면 실험에서 실행할 Python 스크립트 파일을 식별하는 **ScriptRunConfig**를 만든 다음, 이를 기반으로 실험을 실행해야 합니다.

> **참고**: 또한 ScriptRunConfig는 컴퓨팅 대상 및 Python 환경을 결정합니다. 이를 지정하지 않으면 코드가 실행되는 로컬 컴퓨팅(여기서는 이 Notebook이 실행되고 있는 곳)에서 기본 환경이 자동으로 만들어집니다.

다음 셀은 스크립트 기반 실험을 구성하고 제출합니다.

In [ ]:
import os
import sys
from azureml.core import Experiment, ScriptRunConfig
from azureml.widgets import RunDetails


# 스크립트 구성 만들기
script_config = ScriptRunConfig(source_directory=experiment_folder, 
                      script='diabetes_experiment.py') 

# 실험 제출
experiment = Experiment(workspace=ws, name='mslearn-diabetes')
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

이전과 마찬가지로 [Azure Machine Learning Studio](https://ml.azure.com)의 실험 링크나 위젯을 사용해 실험에서 생성된 출력을 확인할 수 있습니다. 또한 실험에서 생성된 파일과 메트릭을 검색하는 코드를 작성할 수도 있습니다.

In [ ]:
# 로깅된 메트릭 가져오기
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

이번에는 실행에서 로그 파일이 생성되었습니다. 이러한 로그 파일은 위젯에서 확인할 수도 있고 앞에서 사용했던 것과 같은 **get_details_with_logs** 메서드를 사용할 수도 있습니다. 이번에는 이 메서드를 사용하면 출력에 로그 데이터가 포함됩니다.

In [ ]:
run.get_details_with_logs()

위의 출력에서도 로그 세부 정보를 확인할 수는 있지만 일반적으로는 로그 파일을 다운로드하여 텍스트 편집기에서 확인하는 방식이 더 쉽습니다.

In [ ]:
import os

log_folder = 'downloaded-logs'

# 모든 파일 다운로드
run.get_all_logs(destination=log_folder)

# 파일이 다운로드되었는지 확인
for root, directories, filenames in os.walk(log_folder): 
    for filename in filenames:  
        print (os.path.join(root,filename))

## 실험 실행 기록 확인

같은 실험을 여러 번 실행했으므로 [Azure Machine Learning Studio](https://ml.azure.com)에서 기록을 확인할 수 있으며 기록된 각 실행을 살펴볼 수도 있습니다. 작업 영역에서 이름으로 실험을 검색하고 SDK를 사용하여 해당 실행을 반복할 수도 있습니다.

In [ ]:
from azureml.core import Experiment, Run

diabetes_experiment = ws.experiments['mslearn-diabetes']
for logged_run in diabetes_experiment.get_runs():
    print('Run ID:', logged_run.id)
    metrics = logged_run.get_metrics()
    for key in metrics.keys():
        print('-', key, metrics.get(key))

## MLflow 사용

MLflow는 기계 학습 프로세스를 관리하기 위한 오픈 소스 플랫폼입니다. MLflow는 전적으로는 아니지만 일반적으로 Databricks 환경에서 실험을 조정하고 메트릭을 추적하는 데 사용됩니다. Azure Machine Learning 실험에서는 기본 로그 기능 대신 MLflow를 사용하여 메트릭을 추적할 수 있습니다.

이 기능을 활용하려면 **mlflow** 및 **azureml-mlflow** 패키지가 필요합니다. 이 두 패키지가 설치되어 있는지 확인해 보겠습니다.

In [ ]:
!pip show mlflow azureml-mlflow

### 인라인 실험에 MLflow 사용

MLflow를 사용하여 인라인 실험의 메트릭을 추적하려면 MLflow *추적 URI*를 실험이 실행 중인 작업 공간으로 설정해야 합니다. 이를 통해 **mlflow** 추적 방법을 사용하여 실험 실행에 데이터를 로그할 수 있습니다.

In [ ]:
from azureml.core import Experiment
import pandas as pd
import mlflow

# 작업 영역으로 MLflow 추적 URI 설정
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

# 작업 영역에서 Azure ML 실험 만들기
experiment = Experiment(workspace=ws, name='mslearn-diabetes-mlflow')
mlflow.set_experiment(experiment.name)

# MLflow 실험 시작
with mlflow.start_run():
    
    print("Starting experiment:", experiment.name)
    
    # 데이터 로드
    data = pd.read_csv('data/diabetes.csv')

    # 행 수를 계산하고 결과 로깅
    row_count = (len(data))
    mlflow.log_metric('observations', row_count)
    print("Run complete")

이제 실행 중에 로깅된 메트릭을 확인해 보겠습니다.

In [ ]:
# 실험의 최신 실행 가져오기
run = list(experiment.get_runs())[0]

# 로깅된 메트릭 가져오기
print("\nMetrics:")
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
    
# Azure ML Studio에서 실험에 대한 링크 받기   
experiment_url = experiment.get_portal_url()
print('See details at', experiment_url)

위의 코드를 실행한 후 표시되는 링크를 사용하여 Azure Machine Learning Studio에서 실험을 볼 수 있습니다. 그런 다음 실험의 최신 실행을 선택하여 **메트릭** 탭을 보고 로그된 메트릭을 확인합니다.

### 실험 스크립트에서 MLflow 사용

MLflow를 사용하여 실험 스크립트의 메트릭을 추적할 수도 있습니다.

다음 두 셀을 실행하여 MLflow를 사용하는 실험을 위한 폴더와 스크립트를 만듭니다.

In [ ]:
import os, shutil

# 실험 파일용 폴더 만들기
folder_name = 'mlflow-experiment-files'
experiment_folder = './' + folder_name
os.makedirs(folder_name, exist_ok=True)

# 실험 폴더에 데이터 파일 복사
shutil.copy('data/diabetes.csv', os.path.join(folder_name, "diabetes.csv"))

In [ ]:
%%writefile $folder_name/mlflow_diabetes.py
from azureml.core import Run
import pandas as pd
import mlflow


# MLflow 실험 시작
with mlflow.start_run():
       
    # 데이터 로드
    data = pd.read_csv('diabetes.csv')

    # 행 수를 계산하고 결과 로깅
    row_count = (len(data))
    print('observations:', row_count)
    mlflow.log_metric('observations', row_count)

Azure ML 실험 스크립트에서 MLflow 추적을 사용하면 실험 실행을 시작할 때 MLflow 추적 URI가 자동으로 설정됩니다. 그러나 스크립트를 실행해야 하는 환경에는 필수 **mlflow** 패키지가 포함되어 있어야 합니다.

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.widgets import RunDetails


# 실험용 Python 환경 만들기
mlflow_env = Environment("mlflow-env")

# 필수 패키지가 설치되어 있는지 확인합니다.
packages = CondaDependencies.create(conda_packages=['pandas','pip'],
                                    pip_packages=['mlflow','azureml-mlflow'])
mlflow_env.python.conda_dependencies = packages

# 스크립트 구성 만들기
script_mlflow = ScriptRunConfig(source_directory=experiment_folder,
                                script='mlflow_diabetes.py',
                                environment=mlflow_env) 

# 실험 제출
experiment = Experiment(workspace=ws, name='mslearn-diabetes-mlflow')
run = experiment.submit(config=script_mlflow)
RunDetails(run).show()
run.wait_for_completion()

평소와 같이 실험이 완료되면 로그된 메트릭을 실험 실행에서 얻을 수 있습니다.

In [ ]:
# 로깅된 메트릭 가져오기
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))

> **추가 정보**: 실험 실행에 대해 자세히 알아보려면 Azure ML 설명서에서 [이 토픽](https://docs.microsoft.com/azure/machine-learning/how-to-manage-runs)을 참조하세요. 실행에서 메트릭을 기록하는 방법에 대한 자세한 내용은 [이 토픽](https://docs.microsoft.com/azure/machine-learning/how-to-track-experiments)을 참조하세요. Azure ML 실험과 MLflow의 통합에 대한 자세한 내용은 [이 토픽](https://docs.microsoft.com/ko-kr/azure/machine-learning/how-to-use-mlflow)을 참조하세요.